In [1]:
import tensorflow as tf

In [2]:
import pandas as pd
import numpy as np

In [3]:
import os
os.getcwd()

'/Users/rthakur/RL_project'

In [4]:
rate_cols = ['item','user','rating','timestamp']
ratings = pd.read_csv('/Users/rthakur/RL_project/data/electronics.csv')
metadata = pd.read_csv('/Users/rthakur/RL_project/data/metadata_df.csv')
ratings.columns = rate_cols

In [5]:
ratings.head()

,item,user,rating,timestamp
0,0060009810,A3P0KRKOBQK1KN,5.0,1025913600
1,0060009810,A192HO2ICJ75VU,5.0,1025654400
2,0060009810,A2T278FKFL3BLT,4.0,1025395200
3,0060009810,A2ZUXVTW8RXBXW,5.0,1025222400
4,0060009810,A21JDG4HA6OLPF,4.0,1024963200


In [6]:
metadata.head()

,category,description,title,brand,feature,date,price,asin
0,"['Electronics', 'Portable Audio & Video', 'MP3...",['Pic N Roll HD-900 5GB MP3 Player with Full C...,5GB MP3 Player with Full Color Viewer,Pic 'N Roll,['PICNROLL MP3 PLAYER W/ 5GB & LCD VIEWER'],"November 21, 2005",NaN,B000E78Y7U
1,"['Electronics', 'Computers & Accessories', 'Co...","[""The ATI Radeon X1900 XTX Pro Graphics Card p...",ATI Radeon X1900 XTX 512 MB PCIE Video Card,ATI,"['512-bit internal ring bus for memory reads',...","January 6, 2006",NaN,B000E78A1K
2,"['Electronics', 'eBook Readers & Accessories',...",[],Kensington 62362 Saddlebag Ultra Notebook Comp...,Kensington Technology,[],NaN,NaN,B000E7AC0W
3,"['Electronics', 'Camera & Photo', 'Accessories...",[],Insten&reg; Replacement for Compatible Recharg...,eForCity,[],"January 25, 2006",NaN,B000E7AKEK
4,"['Electronics', 'Camera & Photo', 'Accessories...",[],NB-3L Battery Charger,Ereplacements,[],"October 2, 2001",NaN,B000E7AKEU


In [47]:
#Finding users who have given more than 1 rating for same item
count_rate = ratings.groupby(['user','item'])['rating'].count().reset_index(name="num_ratings")
wrong_users_df = count_rate.loc[count_rate.num_ratings > 1].reset_index(drop=True)

In [48]:
wrong_users_df.sort_values(by='num_ratings',ascending=False)

,user,item,num_ratings
31164,A19NAKWN2HD289,B001MXQHQ0,41
76106,A1NR1O40OFFWDF,B001MXQHQ0,41
38964,A1C3Y8LKKUBVWY,B001MXQHQ0,41
244500,A34C9AFFZOI45T,B001MXQHQ0,41
166415,A2G0285W12XDM3,B001MXQHQ0,41
...,...,...,...
145752,A29H32DSFBPSL4,B0012TXIC0,2
145751,A29H23SG951OO,B000YBH4YU,2
145750,A29H22A69P1IF8,B00180LGX6,2
145749,A29H1XQ3IFK4TQ,B000XPG2QI,2


In [49]:
merged_df = ratings.merge(wrong_users_df.drop_duplicates(),on='user',how='left',indicator=True)

In [50]:
new_ratings = merged_df.loc[merged_df['_merge'] == 'left_only'][['item_x','user','rating','timestamp']]

In [51]:
new_ratings

,item_x,user,rating,timestamp
0,0060009810,A3P0KRKOBQK1KN,5.0,1025913600
1,0060009810,A192HO2ICJ75VU,5.0,1025654400
2,0060009810,A2T278FKFL3BLT,4.0,1025395200
3,0060009810,A2ZUXVTW8RXBXW,5.0,1025222400
4,0060009810,A21JDG4HA6OLPF,4.0,1024963200
...,...,...,...,...
22648364,B01HJF704M,A1RH0C7YHWZQER,1.0,1493424000
22648365,B01HJF704M,A2955VBJEJZ4S,5.0,1492387200
22648366,B01HJF704M,A1FGCIRPRNZWD5,5.0,1490918400
22648367,B01HJF704M,AOEG7L8HI8DXJ,5.0,1487203200


In [103]:
#Top 15000 items
top_df = new_ratings.groupby('item_x').aggregate(np.count_nonzero).sort_values(by='user',ascending=False).head(n=15000).reset_index()
top_df

,item_x,user,rating,timestamp
0,B010OYASRG,27389,27389,27389
1,B00L0YLRUW,19602,19602,19602
2,B006GWO5WK,15247,15247,15247
3,B00P7EVST6,14786,14786,14786
4,B00ITIQI92,13896,13896,13896
...,...,...,...,...
14995,B00CJHZLEW,209,209,209
14996,B007MX9JBY,209,209,209
14997,B00EMHVVLY,209,209,209
14998,B004X49TAG,209,209,209


In [104]:
df = new_ratings.merge(top_df.drop_duplicates(),on = 'item_x', how = 'left').dropna().reset_index(drop=True)[['item_x','user_x','rating_x','timestamp_x']]
df.columns = rate_cols

In [105]:
df

,item,user,rating,timestamp
0,0764207474,ARJD6V8CA9QQH,5.0,1042416000
1,0764207474,A25J6XS2KNELIS,4.0,1041724800
2,0764207474,A248Z5SXEKZ0IW,5.0,1484697600
3,0764207474,A3RZUL6YXTOI85,5.0,1482883200
4,0764207474,A92UB6ANT7K8U,5.0,1482796800
...,...,...,...,...
10009039,B01HI7O7KQ,AUFTSO0212IOM,1.0,1526947200
10009040,B01HI7O7KQ,AOF1TL7HX03DN,1.0,1526428800
10009041,B01HI7O7KQ,A3N4GYAPP979MR,5.0,1525564800
10009042,B01HI7O7KQ,A1TBJNB3L1QOJ7,1.0,1525392000


In [ ]:
#From below take users with more than 18 ratings 

In [114]:
temp = df.groupby('user')['rating'].count().reset_index()

In [115]:
temp

,user,rating
0,A00010809P09NUU6ZP6H,1
1,A0001392JPQ8N7MSF2CV,1
2,A0001410YRCTKL1MHXQJ,1
3,A0002032ZFQKDVHYKGWR,1
4,A0002686ZNGLCW7JVER2,1
...,...,...
6152953,AZZZVH7FYD0UR,1
6152954,AZZZWQJTEDYBM,1
6152955,AZZZYCYV82JER,2
6152956,AZZZZHRHTLH4R,1


In [116]:
temp_df = df.merge(temp,on='user',how='left').dropna()

In [117]:
temp_df.loc[temp_df.rating_y > 18].reset_index()

,index,item,user,rating_x,timestamp,rating_y
0,470,0972683275,A1MNILX1J1NWV7,4.0,1464825600,25
1,740,0972683275,A1JPB4BJHNCSWR,5.0,1409184000,33
2,993,0972683275,A2A3B0JX2JRS6S,5.0,1379462400,30
3,1257,0972683275,A14O9P3DV5EHKO,3.0,1359331200,19
4,1267,0972683275,A2BLGQ2SCSKCCY,5.0,1358812800,22
...,...,...,...,...,...,...
111242,10008579,B01H4OYJNI,A2QIFKUBK053ZC,5.0,1531008000,23
111243,10008594,B01H4YTHWG,A2SOSDOBI6AHJI,5.0,1537920000,20
111244,10008618,B01H4ZLZLQ,A3CHC46D65IH4H,4.0,1536019200,26
111245,10009025,B01HGV7M7Y,A1F7KIIWHPQ5TG,5.0,1537142400,26
